# 1. Import libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(27)
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn

mean, std = (0.5,), (0.5,)

Creating Pytorch Dataset class to load Kaggle data into Pytorch framework

In [ ]:
# Although Pytorch has MNIST dataset, we will be loading our own MNIST dataset from Kaggle, for this we 
# need to create a custom class as per Pytorch's documentation

class Dataset(object):
    """An abstract class representing a Dataset.
    All other datasets should subclass it. All subclasses should override
    ``__len__``, that provides the size of the dataset, and ``__getitem__``,
    supporting integer indexing in range from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])
    
class CustomImgDataset(Dataset):
    
    def __init__(self, file_path, transform=None, test=False):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        self.test = test
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # we will be using "ToTensor() transform on the input, so our Numpy array must"
        # be of shape (H,W,C) and ToTensor() will convert it to (C,H,W)
        if self.test == True: #test data don't have a label in the dataset
            image = self.data.iloc[index, 0:].values.astype(np.uint8).reshape((28, 28, 1))
            label = 0 #hardcoded to garbage
        else:
            image = self.data.iloc[index, 1:].values.astype(np.uint8).reshape((28, 28, 1))
            label = self.data.iloc[index, 0]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label
    


# 2. Loading Test and Train data and normalizing

In [ ]:
# transforms input to Tensor, and normalize the input elements with the given mean and standard deviation
transform = transforms.Compose ([transforms.ToTensor(), transforms.Normalize(mean,std)])
train_dset = CustomImgDataset('/kaggle/input/digit-recognizer/train.csv', transform=transform)
trainloader = torch.utils.data.DataLoader(train_dset, batch_size=64, shuffle=True)

test_dset = CustomImgDataset('/kaggle/input/digit-recognizer/test.csv', transform=transform, test=True)
testloader = torch.utils.data.DataLoader(test_dset, batch_size=28000, shuffle=False)

# 3. Creating the Model, using CNN, Dropout, MaxPool layers

In [ ]:
class NET(nn.Module):
    def __init__(self):
        super(NET, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        self.drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = self.drop(self.conv2(x))
        x = F.relu(F.max_pool2d(x,2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x
model = NET()

# 4. Setting Loss and Optimizer function

In [ ]:
from torch import optim
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

epochs = 5

for i in range (epochs):
    cum_loss = 0
    
    for images, labels in trainloader:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        cum_loss += loss.item()
    
    print("Epoch: {} Training cumulative loss {}". format(i, cum_loss/len(trainloader)))


Lets try the model on a few test images 

In [ ]:
#Lets take a look at one of the image
def denormalize(tensor):
    tensor = tensor*0.5 + 0.5
    return tensor

def showimg(img):
    i = denormalize(img)
    plt.imshow(i, cmap='gray')

In [ ]:
img, label = next(iter(testloader))
idx = np.random.randint(64, size=1) #pick a random image from the batch
with torch.no_grad():
    output = model(img[idx[0]].view(1,1,28,28)) #run the model on the selected image

pred = torch.exp(output)
pred = pred.numpy()[0]
imgplt = img[idx[0]].view(28,-1)
showimg(imgplt)
predict = np.argmax(pred, 0)
print("Predicted label: ", predict)

Lets generate the Kaggle submission csv

In [ ]:
img, label = next(iter(testloader))
with torch.no_grad():
    output = model(img)
    logoutput = torch.exp(output)
    result = torch.argmax(logoutput,1)
    result = pd.Series(result, name="Label")
    submission = pd.concat([pd.Series(range(1, 28001), name="ImageId"), result], axis=1)
    submission.to_csv("digit-recognizer-submission.csv", index=False)